Проект команды DiaryesGo

In [2]:
from google.colab import files
from IPython.display import clear_output

In [3]:
!pip install pycodestyle pycodestyle_magic
!pip install flake8
clear_output()

In [4]:
%load_ext pycodestyle_magic

In [5]:
!pip install -U -q PyDrive

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [7]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [8]:
whole_table_with_lemm = drive.CreateFile(
    {'id': '1rKrNCmgWPReJujCoZCDlhAb66yiUZ_by'})
whole_table_with_lemm.GetContentFile('whole_table_with_lemm.csv')

In [9]:
import re
import nltk
import joblib

import pandas as pd
import numpy as np

from pprint import pprint
from ast import literal_eval
from datetime import datetime
from string import punctuation

from tqdm.notebook import tqdm

from nltk.corpus import stopwords

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

clear_output()

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
stop_words = set(stopwords.words('russian'))

# Дневниковые записи

In [20]:
def date_parser(date):
    if date and re.match(r'^19\d{2}/[1-9]\d?/[1-9]\d?$', date):
        date_split = [int(elem) for elem in date.split('/')]
        if date_split[0] >= 1900:
            return datetime(year=date_split[0],
                            month=date_split[1],
                            day=date_split[2])
    return np.nan

In [21]:
df = pd.read_csv('/content/whole_table_with_lemm.csv', sep='\t', nrows=484000, index_col=0,
                 usecols=[0, 2, 3], parse_dates=['dates'],
                 date_parser=date_parser).rename(
                     columns={'notes\n': 'lemm', 'dates': 'date'})

In [22]:
df.head(10)

,lemm,date
0,воскресение. \nвчера приезжать слава навеща...,NaT
1,"ночь мы достигать банка антон-дорн, в датский ...",NaT
2,суббота. москва. ехать в селище.\n,NaT
3,18 август его королевский величество отправлят...,NaT
4,"на следующий день, 19 август, прибывать в остр...",NaT
5,"20 к ночь, прибыль в сморгонь.\n",NaT
6,"21 к ночь, прибывать в марх.\n",NaT
7,"22, к ночь, доходить до молодочный, куда прибы...",NaT
8,"23 к ночь, прибыль в радошкович.\n",NaT
9,"24 к ночь, в соломержич.\n",NaT


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384526 entries, 0 to 384525
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   lemm    384525 non-null  object        
 1   date    315483 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 8.8+ MB


In [24]:
!rm whole_table_with_lemm.csv

In [25]:
df = df.dropna().set_index('date')

In [26]:
df.head()

,lemm
date,
1900-01-01,мраморный. \nвоскресение. 19 декабрь. в 9 ...
1900-01-01,понедельник. с восемь час литургия. служить о....
1900-01-01,"*понедельник.* оттепель продолжаться, как всег..."
1900-01-01,*20.* жена поправляться.\n
1900-01-01,"мой брат александр, изучать право в здешний ун..."


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 315482 entries, 1900-01-01 to 1999-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   lemm    315482 non-null  object
dtypes: object(1)
memory usage: 4.8+ MB


In [28]:
punctuation + '— \n'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~— \n'

In [29]:
regex = r'[!\"#$%&\'()*+,\-./:;<=>?@[\\\]^_`{|}~— \n]+'

In [30]:
def tokenization(text):
    return [elem for elem in re.split(regex, text)
            if elem and elem not in stop_words]

In [31]:
df['lemm'] = df['lemm'].apply(tokenization)
clear_output()

In [32]:
df.head()

,lemm
date,
1900-01-01,"[мраморный, воскресение, 19, декабрь, 9, 1, 2,..."
1900-01-01,"[понедельник, восемь, час, литургия, служить, ..."
1900-01-01,"[понедельник, оттепель, продолжаться, сильный,..."
1900-01-01,"[20, жена, поправляться]"
1900-01-01,"[брат, александр, изучать, право, здешний, уни..."


In [33]:
df.to_csv('texts.csv')

In [34]:
!zip texts.zip texts.csv

  adding: texts.csv (deflated 78%)


In [35]:
!rm texts.csv

In [36]:
files.download('texts.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# код для загрузки готовой к topic modelling таблицы

# prozhito_lemmas = drive.CreateFile(
#    {'id': '1_t6vdR2iREGMdNNtuYMc09anV0YpbaSs'})
# prozhito_lemmas.GetContentFile('texts.zip')

In [37]:
def data_to_list(filename):
    df = pd.read_csv(filename)
    df['lemm'] = df['lemm'].apply(literal_eval)
    clear_output()
    return df

## Разбиение на периоды
Так как 99% часть текстов корпуса приходится на советскую эпоху, решено было выделить следующие периоды:
*   *1900-1910*(первая революция)
*   *1910-1914*(перед войной)
*   *1914-1920* (Первая мировая война и революция)
*   *1920-1930* (гражданская война, ссср)
*   *1930-1941* (перед войной)
*   *1941-1945* (война)
*   *1945-1968* (оттепель)
*   *1968-1990* (застой и горбачев)
*   *1990 – 2000* (развал ссср и новая россия)

In [38]:
df = [
    df.loc[
        pd.Timestamp('1905-1-1 00:00:00'): pd.Timestamp('1907-1-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1907-2-1 00:00:00'): pd.Timestamp('1914-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1914-6-1 00:00:00'): pd.Timestamp('1918-10-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1918-11-1 00:00:00'): pd.Timestamp('1927-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1928-1-1 00:00:00'): pd.Timestamp('1939-8-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1939-9-1 00:00:00'): pd.Timestamp('1945-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1946-1-1 00:00:00'): pd.Timestamp('1963-12-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1964-1-1 00:00:00'): pd.Timestamp('1987-5-31 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1987-6-1 00:00:00'): pd.Timestamp('1990-6-30 23:59:59')
        ],
    df.loc[
        pd.Timestamp('1991-1-1 00:00:00'): pd.Timestamp('2001-12-31 23:59:59')]
       ]

In [40]:
df[5].head()

,lemm
date,
1939-09-01,"[сегодня, открытие, большой, миша, пойти, туда..."
1939-09-01,"[морской, ванна, 9, вечер, кинофильм, «комсомо..."
1939-09-01,"[пятница, просыпаться, 9, ч, приходить, юрка, ..."
1939-09-01,"[война, объявление, это, неважный, гитлер, нап..."
1939-09-01,"[сентябрь, 1939, г, осень, остроскучный, хмуры..."


In [42]:
df[5]['lemm'][0]

['сегодня',
 'открытие',
 'большой',
 'миша',
 'пойти',
 'туда',
 'часок',
 'приходить',
 'борис']

In [43]:
period = ['first revolution', 'between revolutions', 'WWI', 'civil war',
          'great purge', 'WWII', 'before ottepel', 'stagnation','perestroyka',
          '90s']

In [44]:
for i, time in enumerate(period):
    print(f'{time}:{" " * (20 - len(time))}\t{df[i].shape[0]} entries')

first revolution:    	5821 entries
between revolutions: 	18761 entries
WWI:                 	20362 entries
civil war:           	35981 entries
great purge:         	37175 entries
WWII:                	62686 entries
before ottepel:      	39742 entries
stagnation:          	58121 entries
perestroyka:         	6723 entries
90s:                 	13283 entries


Сохраняем и загружаем на гуглдиск разбитые на периоды данные.

In [45]:
for i, table in tqdm(enumerate(df)):
    table.to_csv(f'{i + 1}_{period[i].replace(" ", "_")}.csv')

0it [00:00, ?it/s]

In [47]:
!zip periods.zip *.csv

  adding: 10_90s.csv (deflated 78%)
  adding: 1_first_revolution.csv (deflated 78%)
  adding: 2_between_revolutions.csv (deflated 78%)
  adding: 3_WWI.csv (deflated 78%)
  adding: 4_civil_war.csv (deflated 77%)
  adding: 5_great_purge.csv (deflated 78%)
  adding: 6_WWII.csv (deflated 78%)
  adding: 7_before_ottepel.csv (deflated 78%)
  adding: 8_stagnation.csv (deflated 78%)
  adding: 9_perestroyka.csv (deflated 78%)


In [48]:
files.download('periods.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Моделирование

In [50]:
models = []

In [51]:
for table in df[8:]:
    data = list(table['lemm'])
    id2word = Dictionary(data)
    corpus = [id2word.doc2bow(text) for text in data]
    model = LdaModel(corpus, num_topics=10, id2word=id2word, passes=15,
                     iterations=100, per_word_topics=True)
    models.append(model)
    clear_output()
    print(f'Обучено моделей: {len(models)}')

Обучено моделей: 2


In [52]:
joblib.dump(models, 'models.pkl')

['models.pkl']

In [53]:
!zip models.zip models.pkl

  adding: models.pkl (deflated 52%)


In [54]:
files.download('models.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

1900-1910

In [55]:
pprint(models[0].print_topics())

[(0,
  '0.005*"астраханский" + 0.003*"г" + 0.003*"вика" + 0.003*"астрахань" + '
  '0.003*"монета" + 0.003*"л" + 0.003*"луг" + 0.002*"клад" + 0.002*"век" + '
  '0.002*"берег"'),
 (1,
  '0.007*"весь" + 0.005*"машина" + 0.004*"самолет" + 0.004*"час" + 0.004*"это" '
  '+ 0.003*"сидеть" + 0.003*"ночь" + 0.003*"госпиталь" + 0.003*"экипаж" + '
  '0.003*"полет"'),
 (2,
  '0.013*"это" + 0.007*"весь" + 0.007*"который" + 0.006*"горбачев" + '
  '0.006*"свой" + 0.005*"наш" + 0.005*"год" + 0.004*"съезд" + 0.004*"м" + '
  '0.004*"человек"'),
 (3,
  '0.025*"id" + 0.025*"com" + 0.007*"это" + 0.006*"русский" + 0.006*"стих" + '
  '0.005*"очень" + 0.005*"читать" + 0.005*"свой" + 0.005*"книга" + '
  '0.004*"писать"'),
 (4,
  '0.009*"…" + 0.008*"река" + 0.005*"р" + 0.005*"dur" + 0.004*"слушать" + '
  '0.004*"берег" + 0.004*"маршрут" + 0.003*"дорога" + 0.003*"след" + '
  '0.003*"№"'),
 (5,
  '0.012*"это" + 0.009*"весь" + 0.009*"человек" + 0.008*"свой" + 0.007*"жизнь" '
  '+ 0.006*"день" + 0.005*"наш" + 0.005

1910-1914

In [58]:
pprint(models[1].print_topics())

[(0,
  '0.007*"гойя" + 0.006*"религия" + 0.006*"бытие" + 0.005*"тайна" + '
  '0.004*"культура" + 0.004*"дух" + 0.004*"музыка" + 0.004*"русский" + '
  '0.004*"достоевский" + 0.003*"личность"'),
 (1,
  '0.009*"дом" + 0.007*"город" + 0.006*"наш" + 0.006*"который" + 0.005*"улица" '
  '+ 0.005*"идти" + 0.004*"вода" + 0.004*"место" + 0.004*"большой" + '
  '0.004*"дорога"'),
 (2,
  '0.009*"день" + 0.007*"весь" + 0.005*"это" + 0.005*"свой" + 0.005*"который" '
  '+ 0.005*"утро" + 0.004*"мама" + 0.004*"год" + 0.004*"сказать" + '
  '0.004*"наш"'),
 (3,
  '0.010*"физ" + 0.009*"English" + 0.004*"ватник" + 0.004*"проездной" + '
  '0.003*"балагур" + 0.003*"бал" + 0.002*"1995" + 0.002*"тамошний" + '
  '0.002*"ото" + 0.002*"экз"'),
 (4,
  '0.018*"это" + 0.010*"весь" + 0.005*"свой" + 0.005*"мочь" + 0.005*"жизнь" + '
  '0.004*"время" + 0.004*"наш" + 0.003*"давать" + 0.003*"становиться" + '
  '0.003*"работа"'),
 (5,
  '0.017*"театр" + 0.009*"любимов" + 0.007*"спектакль" + 0.006*"сцена" + '
  '0.005*"играт

1914-1920

In [59]:
pprint(models[2].print_topics())

IndexError: ignored

1920-1930

In [ ]:
pprint(models[3].print_topics())

1930-1941

In [ ]:
pprint(models[4].print_topics())

1941-1945

In [ ]:
pprint(models[5].print_topics())

1945-1968

In [ ]:
pprint(models[6].print_topics())

1968-1990

In [ ]:
pprint(models[7].print_topics())

1990-2000

In [ ]:
pprint(models[8].print_topics())